In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
plt.style.use('fivethirtyeight')

pd.options.display.max_columns = 100

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('input/train.csv')
data_test = pd.read_csv('input/test.csv')

# Базовый анализ данных их train.csv

* info показывает что есть пропуски только в колонках LifeSquare и Healthcare_1

In [3]:
data.head(10)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011,0.309479,B,B,35,7715,4,990.0,0,6,B,215898.447742
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,0.460556,B,B,20,4386,14,NaN,1,5,B,296021.204377
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,229102.795999
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,0.075779,B,B,6,1437,3,NaN,0,2,B,95380.220993


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


* describe показывает, что есть проблемы с колонками KitchenSquare (min, max), Rooms (min, max), Square(min, max), LifeSquare (min, max)

In [5]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


## DistrictId

* Для поля DistrictId можно, следуя примеру, рассмотренному на уроке, создать дополнительное поле со средними ценами

## Rooms

In [6]:
data.loc[data['Rooms'] > 5, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,NaN,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,NaN,0,1,B,78364.616704
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,1322.0,3,8,B,229661.964416
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,30.0,2,4,B,172329.270863


In [7]:
data.loc[data['Rooms'] < 1, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,NaN,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,NaN,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1183.0,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,NaN,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,NaN,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,219281.918007


## Square

* присутствуют квартиры с значением в Square < 10

In [8]:
data.loc[data['Square'] < 20, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
212,1748,88,2.0,5.497061,67.628717,1.0,24,22.0,1977,0.127376,B,B,43,8429,3,NaN,3,9,B,412511.088764
1316,11526,27,1.0,4.633498,1.969969,1.0,18,1.0,1977,0.011654,B,B,4,915,0,NaN,0,0,B,107604.269441
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B,137597.601458
1891,13685,0,1.0,17.498542,13.446544,0.0,7,5.0,1961,0.319809,B,B,25,4756,16,2857.0,5,8,B,88871.750230
3166,15127,150,1.0,17.046188,20.036232,8.0,7,12.0,1982,0.300323,B,B,52,10311,6,NaN,1,9,B,74222.110465
3280,10527,27,1.0,4.380726,40.805837,1.0,10,17.0,2013,0.211401,B,B,9,1892,0,NaN,0,1,B,97560.720383
3413,9487,5,1.0,5.129222,5.549458,1.0,1,1.0,1977,0.150818,B,B,16,3433,4,2643.0,4,5,B,369472.403061
3794,523,101,1.0,18.509177,15.170851,3.0,5,5.0,1970,0.225825,A,B,41,6558,7,1548.0,3,2,B,115845.861527
4739,12676,81,3.0,13.784865,15.988889,7.0,4,5.0,1960,0.319809,B,B,25,4756,16,2857.0,5,8,B,78388.806186
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,NaN,0,0,B,126596.941798


## LifeSquare
* присутствуют NaN значения
* присутствуют 3-комнатные квартиры с жилой площадью > 1000
* в некоторый квартирах значение в поле LifeSquare > значения в поле Square, что по сути абсурд

In [9]:
data.loc[data['LifeSquare'] > 8000]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price


In [10]:
data.loc[data['LifeSquare'] > data['Square']].head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
28,8054,23,1.0,42.530043,43.967759,1.0,3,9.0,2014,0.034656,B,B,0,168,0,NaN,0,0,B,95338.198549
44,10521,38,3.0,104.211396,106.340403,0.0,20,0.0,2017,0.060753,B,B,15,2787,2,520.0,0,7,B,435462.048070
52,2301,1,2.0,61.400054,65.224603,0.0,17,22.0,2016,0.007122,B,B,1,264,0,NaN,0,1,B,199215.452229
123,8753,25,3.0,85.952306,89.803753,1.0,4,3.0,2017,0.069753,B,B,53,13670,4,NaN,1,11,B,309688.592681
153,9870,62,1.0,51.831473,53.491301,1.0,5,1.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,131797.472284


In [11]:
data.loc[data['LifeSquare'] > data['Square']].shape

(482, 20)

## KitchenSquare

* присутствуют квартиры с значениями в поле KitchenSquare > 1000
* присутствуют квартиры с значениями в поле KitchenSquare > Square или LifeSquare

In [12]:
data.loc[data['KitchenSquare'] > 1000, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,NaN,0,0,A,108337.484207
8584,14679,81,1.0,32.276663,19.278394,1970.0,6,1.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,105539.556275


In [13]:
data.loc[data['KitchenSquare'] > data['Square'], :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,NaN,0,0,A,108337.484207
5149,13703,42,1.0,38.071692,19.723548,73.0,9,10.0,2006,0.158249,B,B,21,5731,0,NaN,1,0,B,160488.033165
7088,6569,27,1.0,38.220258,18.716856,84.0,4,17.0,2018,0.011654,B,B,4,915,0,NaN,0,0,B,99079.960518
8584,14679,81,1.0,32.276663,19.278394,1970.0,6,1.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,105539.556275


In [14]:
data.loc[data['KitchenSquare'] > data['LifeSquare'], :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
227,16395,2,3.0,79.722243,44.731219,72.0,12,16.0,1987,0.130618,B,B,39,10418,9,900.0,1,9,B,370148.625285
551,1315,6,1.0,48.128114,45.023531,48.0,21,1.0,2013,0.243205,B,B,5,1564,0,540.0,0,0,B,146950.912410
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,NaN,0,0,A,108337.484207
1369,2371,27,2.0,68.841073,64.234956,66.0,4,2.0,2014,0.017647,B,B,2,469,0,NaN,0,0,B,189244.249909
1800,13113,27,1.0,37.805231,2.228592,11.0,6,0.0,2015,0.017647,B,B,2,469,0,NaN,0,0,B,137402.092529
1860,4265,161,2.0,53.216778,32.644859,53.0,7,17.0,1994,0.000699,B,B,14,3369,24,4129.0,0,3,B,261125.669724
1972,15691,62,1.0,41.735073,12.423180,14.0,3,15.0,2010,0.072158,B,B,2,629,1,NaN,0,0,A,80331.107871
4079,6508,23,2.0,67.146049,33.959154,63.0,5,17.0,2019,0.034656,B,B,0,168,0,NaN,0,0,B,193130.585871
4588,448,48,1.0,41.186904,1.626502,41.0,2,1.0,1977,0.041125,B,B,46,9515,5,NaN,1,10,B,216882.265408
5149,13703,42,1.0,38.071692,19.723548,73.0,9,10.0,2006,0.158249,B,B,21,5731,0,NaN,1,0,B,160488.033165


## HouseYear

In [15]:
data.loc[data['HouseYear'] > 2020, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,B,B,30,6141,10,262.0,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,B,B,25,4756,16,2857.0,5,8,B,243028.603096


## Healthcare_1

* можно увидеть, что поле Healthcare_1 практически полностью (за редким исключением) коррелирует с полем Social_2
поэтому, на мой взгляд, его можно удалить. 

In [16]:
# получим в отдельный датасет данные, где Healthcare_1 имеет значение.
data_hc1_not_null = data.loc[data['Healthcare_1'].notnull(), :]

# выведем значения Social_2 для всех Healthcare_1
for hc1 in data['Healthcare_1'].unique():
    print(f'HC1 value: {hc1}')
    print(data_hc1_not_null.loc[data_hc1_not_null['Healthcare_1'] == hc1, :]['Social_2'].unique())

HC1 value: nan
[]
HC1 value: 240.0
[10309  5261]
HC1 value: 229.0
[7759]
HC1 value: 1084.0
[5735]
HC1 value: 2078.0
[5776]
HC1 value: 990.0
[7715]
HC1 value: 1183.0
[6207]
HC1 value: 1406.0
[11395]
HC1 value: 2300.0
[11217  5664]
HC1 value: 2857.0
[4756]
HC1 value: 645.0
[5285]
HC1 value: 1340.0
[5613]
HC1 value: 2301.0
[7077]
HC1 value: 1036.0
[4378]
HC1 value: 1937.0
[5288]
HC1 value: 2890.0
[3079]
HC1 value: 2672.0
[6714]
HC1 value: 1322.0
[10573]
HC1 value: 1548.0
[6558]
HC1 value: 520.0
[2787]
HC1 value: 125.0
[8004]
HC1 value: 1450.0
[4472]
HC1 value: 2643.0
[3433]
HC1 value: 540.0
[1564]
HC1 value: 750.0
[7065]
HC1 value: 830.0
[5889]
HC1 value: 1994.0
[2398]
HC1 value: 3855.0
[4625]
HC1 value: 30.0
[ 4798  5648 10621]
HC1 value: 1575.0
[5148]
HC1 value: 730.0
[5011]
HC1 value: 4087.0
[4789]
HC1 value: 1145.0
[11319]
HC1 value: 1786.0
[1660]
HC1 value: 165.0
[4346]
HC1 value: 628.0
[3293]
HC1 value: 176.0
[7687]
HC1 value: 192.0
[3594]
HC1 value: 46.0
[5027]
HC1 value: 3300.0
[4

In [17]:
data.loc[data['Social_2'] == 11217.0, :].head(20)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
15,2823,85,3.0,73.612225,53.892089,6.0,4,9.0,1979,0.037178,B,B,52,11217,1,2300.0,1,7,B,89084.327876
77,2435,85,4.0,65.559834,47.696255,6.0,6,9.0,1976,0.037178,B,B,52,11217,1,2300.0,1,7,B,266989.016142
135,3749,85,1.0,42.198946,20.794836,10.0,16,22.0,1987,0.037178,B,B,52,11217,1,2300.0,1,7,B,87182.523385
159,15585,85,1.0,44.038524,40.435804,1.0,15,22.0,1982,0.037178,B,B,52,11217,1,2300.0,1,7,B,202321.231473
219,5354,85,1.0,28.535084,12.733642,6.0,4,9.0,1967,0.037178,B,B,52,11217,1,2300.0,1,7,B,142269.268090
417,11886,85,3.0,53.530113,37.011747,6.0,8,9.0,1980,0.037178,B,B,52,11217,1,2300.0,1,7,B,117408.423659
638,3154,85,1.0,72.998799,NaN,12.0,13,17.0,1977,0.037178,B,B,52,11217,1,2300.0,1,7,B,374073.836779
733,15510,85,2.0,61.621345,36.447372,10.0,14,22.0,1988,0.037178,B,B,52,11217,1,2300.0,1,7,B,288447.830966
741,15690,85,2.0,57.259748,36.178666,9.0,6,16.0,1976,0.037178,B,B,52,11217,1,2300.0,1,7,B,268393.878152
914,10402,85,3.0,62.085306,46.337568,6.0,10,9.0,1979,0.037178,B,B,52,11217,1,2300.0,1,7,B,118038.561587


# Изменяем данные

In [18]:
train, valid = train_test_split(data, test_size=0.2, random_state=42)

In [19]:
train_test_merged = pd.concat([data[['Rooms', 'Square', 'LifeSquare']], data_test[['Rooms', 'Square', 'LifeSquare']]])


In [20]:
rooms_mean_square = train_test_merged.groupby(['Rooms'], as_index=False)[['Square']].mean()\
    .rename(columns={'Square':'mean_square'})
rooms_mean_square = rooms_mean_square[rooms_mean_square['Rooms'] < 6]
rooms_mean_square = rooms_mean_square[rooms_mean_square['Rooms'] > 0]
rooms_mean_square


,Rooms,mean_square
1,1.0,41.200432
2,2.0,56.754185
3,3.0,77.001488
4,4.0,96.923976
5,5.0,120.703670


In [21]:
rooms_mean_lifesquare = train_test_merged.groupby(['Rooms'], as_index=False)[['LifeSquare']].mean()\
    .rename(columns={'LifeSquare':'mean_lifesquare'})
rooms_mean_lifesquare = rooms_mean_lifesquare[rooms_mean_lifesquare['Rooms'] < 6]
rooms_mean_lifesquare = rooms_mean_lifesquare[rooms_mean_lifesquare['Rooms'] > 0]
rooms_mean_lifesquare 

,Rooms,mean_lifesquare
1,1.0,24.996356
2,2.0,36.057513
3,3.0,53.353981
4,4.0,65.396877
5,5.0,81.456484


In [22]:
def add_mean_square(df, rms=rooms_mean_square, rmls=rooms_mean_lifesquare):
    df = pd.merge(df, rms, on=['Rooms'], how='left')
    df = pd.merge(df, rmls, on=['Rooms'], how='left')
    return df

In [23]:
def clean_rooms(df):
    df.loc[df['Rooms'] > 5, 'Rooms'] = 5
    df.loc[df['Rooms'] < 1, 'Rooms'] = 1
    return df

In [24]:
def clean_square(df):
    df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'])
    df.loc[df['Square'] < 20, 'Square'] = df['LifeSquare'] + 6
    df.loc[df['Square'] < 20, 'Square'] = 20
    df.loc[df['LifeSquare'] > df['Square'], 'LifeSquare'] = df['Square'] - 6
    
    # хотел тут использовать средние значения жилой и общей площади, но не успел. 
    # поэтому оставляю так.
        
    return df

In [25]:
def clean_year(df):
    df = df[df['HouseYear'] < 2020]
    df = df[df['HouseYear'] > 1900]
    return df

In [26]:
stat1 = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price1'})

In [27]:
stat2 = train.groupby('Rooms', as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price2'})

In [28]:
mean_price = train['Price'].mean()

In [29]:
def add_mean_price(df, stat1, stat2, mean_price):
    df = pd.merge(df, stat1, on=['DistrictId', 'Rooms'], how='left')
    df = pd.merge(df, stat2, on='Rooms', how='left')
    
    df['mean_price2'] = df['mean_price2'].fillna(mean_price)
    df['mean_price1'] = df['mean_price1'].fillna(df['mean_price2'])
    return df

In [30]:
def prepare_data(df, stat1=stat1, stat2=stat2, mean_price=mean_price):
    df = clean_rooms(df)
    df = clean_year(df)
    df = add_mean_price(df, stat1, stat2, mean_price)
    df = add_mean_square(df, rooms_mean_square, rooms_mean_lifesquare)
    df = clean_square(df)
    df = pd.get_dummies(df)

    return df

In [31]:
train = prepare_data(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7981 entries, 0 to 7980
Data columns (total 27 columns):
Id                 7981 non-null int64
DistrictId         7981 non-null int64
Rooms              7981 non-null float64
Square             7981 non-null float64
LifeSquare         7981 non-null float64
KitchenSquare      7981 non-null float64
Floor              7981 non-null int64
HouseFloor         7981 non-null float64
HouseYear          7981 non-null int64
Ecology_1          7981 non-null float64
Social_1           7981 non-null int64
Social_2           7981 non-null int64
Social_3           7981 non-null int64
Healthcare_1       4136 non-null float64
Helthcare_2        7981 non-null int64
Shops_1            7981 non-null int64
Price              7981 non-null float64
mean_price1        7981 non-null float64
mean_price2        7981 non-null float64
mean_square        7981 non-null float64
mean_lifesquare    7981 non-null float64
Ecology_2_A        7981 non-null uint8
Ecology_2_B

In [32]:
valid = prepare_data(valid)

In [33]:
train.head(10)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,mean_price1,mean_price2,mean_square,mean_lifesquare,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,12473,58,1.0,65.271225,62.798045,1.0,3,1.0,1977,0.437885,23,5735,3,1084.0,0,5,210849.693542,126758.726959,159627.216569,41.200432,24.996356,0,1,0,1,0,1
1,14050,146,2.0,45.091598,43.816601,1.0,5,18.0,2017,0.236108,16,3893,27,NaN,3,10,201549.998077,250144.415850,215356.907285,56.754185,36.057513,0,1,0,1,0,1
2,3779,5,1.0,34.463114,20.561823,5.0,3,5.0,1960,0.150818,16,3433,4,2643.0,4,5,158433.168775,240410.958424,159627.216569,41.200432,24.996356,0,1,0,1,0,1
3,3762,90,3.0,61.931107,46.126389,5.0,4,5.0,1966,0.265089,37,5288,0,1937.0,3,2,220194.818430,340638.806841,290307.800100,77.001488,53.353981,0,1,0,1,0,1
4,358,1,1.0,42.670840,42.670840,1.0,8,17.0,1977,0.007122,1,264,0,NaN,0,1,125834.270291,146422.288705,159627.216569,41.200432,24.996356,0,1,0,1,0,1
5,6519,94,2.0,50.392833,3.249569,1.0,4,12.0,1973,0.521867,25,6149,0,NaN,0,0,157822.338178,209457.238445,215356.907285,56.754185,36.057513,0,1,0,1,0,1
6,6076,27,1.0,48.599513,48.599513,1.0,8,1.0,1977,0.072158,2,629,1,NaN,0,0,159768.113188,122862.116648,159627.216569,41.200432,24.996356,0,1,0,1,1,0
7,12584,23,1.0,41.515204,41.515204,1.0,12,25.0,1977,0.075779,6,1437,3,NaN,0,2,118200.065926,101899.009397,159627.216569,41.200432,24.996356,0,1,0,1,0,1
8,4144,23,1.0,42.705836,42.705836,0.0,5,4.0,1977,0.034656,0,168,0,NaN,0,0,92290.968476,101899.009397,159627.216569,41.200432,24.996356,0,1,0,1,0,1
9,15303,48,3.0,83.866174,83.866174,1.0,26,1.0,2014,0.041125,46,9515,5,NaN,1,10,468595.039049,390418.989467,290307.800100,77.001488,53.353981,0,1,0,1,0,1


In [34]:
valid.head(10)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,mean_price1,mean_price2,mean_square,mean_lifesquare,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,3702,21,2.0,48.812195,34.679779,5.0,5,5.0,1963,0.034331,25,5487,3,NaN,0,4,181530.459031,189415.181141,215356.907285,56.754185,36.057513,0,1,0,1,0,1
1,12848,61,3.0,81.103039,49.310278,8.0,4,4.0,1960,0.298205,16,4048,3,NaN,1,3,260456.004692,275623.728869,290307.800100,77.001488,53.353981,0,1,0,1,0,1
2,2239,6,3.0,82.882978,3.977650,1.0,8,17.0,1977,0.243205,5,1564,0,540.0,0,0,219945.304640,213456.026157,290307.800100,77.001488,53.353981,0,1,0,1,0,1
3,15611,23,1.0,33.863324,29.993297,0.0,5,4.0,1977,0.034656,0,168,0,NaN,0,0,66883.280318,101899.009397,159627.216569,41.200432,24.996356,0,1,0,1,0,1
4,5634,52,1.0,43.095135,43.095135,1.0,10,17.0,1977,0.371149,34,7065,1,750.0,2,5,114086.065201,135032.304603,159627.216569,41.200432,24.996356,0,1,0,1,0,1
5,14726,53,3.0,84.569998,60.613297,12.0,18,24.0,2017,0.049637,34,7759,0,229.0,1,3,359065.287230,343809.126120,290307.800100,77.001488,53.353981,0,1,0,1,0,1
6,7247,69,2.0,54.663924,29.266790,9.0,13,17.0,1999,0.000991,33,6783,3,NaN,1,9,377445.078933,313565.297439,215356.907285,56.754185,36.057513,0,1,0,1,0,1
7,2018,94,1.0,33.521641,30.155171,1.0,17,16.0,2015,0.282798,33,8667,2,NaN,0,6,381912.162109,157420.842578,159627.216569,41.200432,24.996356,0,1,0,1,0,1
8,14587,6,1.0,40.696294,34.696294,1.0,4,17.0,2014,0.243205,5,1564,0,540.0,0,0,133072.279471,135955.411695,159627.216569,41.200432,24.996356,0,1,0,1,0,1
9,3643,46,2.0,44.369058,31.064926,5.0,1,5.0,1965,0.188784,31,6137,4,NaN,0,1,184036.005146,227126.750387,215356.907285,56.754185,36.057513,0,1,0,1,0,1


# Model

In [35]:
from sklearn.ensemble import RandomForestRegressor as RF

In [36]:
from sklearn.metrics import r2_score as r2

In [37]:
# после того, как я убрал средние цены из модели и добавил обратно DistrictID без обработки,
# дела пошли сильно лучше :) 
feats = ['Square', 'Rooms', 'LifeSquare', 'HouseFloor', 'HouseYear', 'Ecology_1',
         'Social_1', 'Social_2', 'Social_3', 'mean_square', 'mean_lifesquare', 'DistrictId']




In [38]:
model = RF(n_estimators=1280, max_depth=20, random_state=42)
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1280, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [39]:
def evaluate_model(df, model=model, feats=feats, calculate_r2='Yes'):
    pred = model.predict(df.loc[:, feats])
    if calculate_r2 == 'Yes':
        r2_ = r2(df['Price'], pred)
        print('R2: {}'.format(r2_))
    return pred

In [40]:
pred_train = evaluate_model(train)

R2: 0.9607433808610176


In [41]:
pred_valid = evaluate_model(valid)

R2: 0.7146187113688895


In [42]:
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feats, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
for pair in feature_importances:
    print('Feature: {:20} Importance: {}'.format(*pair)) 

Feature: Square               Importance: 0.4
Feature: Social_1             Importance: 0.1
Feature: Social_2             Importance: 0.1
Feature: Social_3             Importance: 0.07
Feature: DistrictId           Importance: 0.06
Feature: LifeSquare           Importance: 0.05
Feature: HouseYear            Importance: 0.05
Feature: Ecology_1            Importance: 0.05
Feature: Rooms                Importance: 0.03
Feature: HouseFloor           Importance: 0.03
Feature: mean_square          Importance: 0.03
Feature: mean_lifesquare      Importance: 0.03


## Test

In [43]:
test = pd.read_csv('input/test.csv')

In [44]:
test = prepare_data(test)

In [45]:
test['Price'] = evaluate_model(test, calculate_r2='No')

In [46]:
test['Price'].head(10)

0    158264.247615
1    226798.106995
2    160943.638635
3    341145.404154
4    138850.314413
5    207806.113216
6    171535.056761
7    228752.438379
8    294697.166106
9    213335.964497
Name: Price, dtype: float64

In [47]:
test.loc[:, ['Id', 'Price']].to_csv('SArlashin_predictions.csv', index=None)